# Проект 2. Анализ для модели кредитного скоринга

**Цель исследования** - проверить:

Влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.

**Задачи исследования** - проверить:

* Есть ли зависимость между количеством детей и возвратом кредита в срок?
* Есть ли зависимость между семейным положением и возвратом кредита в срок?
* Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
* Как разные цели кредита влияют на его возврат в срок?

**Ход исследования**
1. Обзор данных
2. Предобработка данных
3. Анализ данных и решение поставленных задач
4. Итоги исследования

## Обзор данных

Импортируем библиотку pandas

In [1]:
import pandas as pd

Сохраняем файл данных в переменную data

In [2]:
try:
    data = pd.read_csv('D:\Documents\Python\ЯПрактикум\Проекты\Проект 2 Кредиты\data.csv') # путь к локальному файлу данных
except: 
    data = pd.read_csv('/datasets/data.csv') # путь к файлу данных на сервере ЯПрактикума

Выводим на экран первые 10 строк таблицы. 

Сразу заметна проблема с наличием отрицательных значений в столбце "days_employed", а также с дробными значениями в столбце total_income. Вернемся к ним позднее на этапе предобработки данных.

In [3]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Получаем общую информацию о таблице методом info()

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В таблице 12 столбцов, столбцы названы без нарушений стиля. 

Согласно документации к проекту:

* children — количество детей в семье
* days_employed — общий трудовой стаж в днях
* dob_years — возраст клиента в годах
* education — уровень образования клиента
* education_id — идентификатор уровня образования
* family_status — семейное положение
* family_status_id — идентификатор семейного положения
* gender — пол клиента
* income_type — тип занятости
* debt — имел ли задолженность по возврату кредитов
* total_income — ежемесячный доход
* purpose — цель получения кредита

В 2 столбцах обнаруживаются пропуски в данных, данную проблему будет необходимо решить на этапе предобработки данных.

**Выводы**

Предварительно можно утверждать, что данных достаточно для решения поставленных задач. Однако перед проведением анализа необходимо выполнить предобработку данных:

1. Оценить критичность и заполнить пропущенные значения в столбцах days_employed, total_income
2. Найти причину и исправить отрицательные значения в столбце days_employed
3. Округлить значения дохода в столбце total_income, приведя его к целочисленному типу данных
4. Проверить наличие и устранить дубликаты в данных (явные и неявные)
5. Провести необходимые группировки данных для повышения удобства и наглядности дальнейшего анализа 

## Предобработка данных

### Пропуски значений

Посчитаем долю пропущенных значений в таблице:

In [5]:
data.isna().sum()/data['children'].count()

children            0.000000
days_employed       0.100999
dob_years           0.000000
education           0.000000
education_id        0.000000
family_status       0.000000
family_status_id    0.000000
gender              0.000000
income_type         0.000000
debt                0.000000
total_income        0.100999
purpose             0.000000
dtype: float64

В столбцах days_employed и total_income пропущено по 2174 значений, что составляет 10% от общего количества значений в столбцах. Доля пропущенных значений довольно существенна и может повлиять на результаты исследования.

Чтобы понять причины пропусков данных взглянем на несколько строк с пропущенными значениями:

In [6]:
data[data['days_employed'].isna() == True].head() # фильтрация через метод isna и логическую индексацию

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


Вначале можно было бы предположить, что значения в столбцах days_employed и total_income могут отсутствовать по причине того, что заемщики пока не успели получить трудовой стаж и находятся в поисках первого места работы. 

Однако при рассмотрении датасета с пропусками видно, что это не так - возраст заемщиков и тип дохода говорит о том, что скорее всего у них был/есть опыт работы и причина пропусков вероятнее всего заключается в том, что заемщики не предоставили данные о своем трудовом стаже и доходе, либо произошла техническая ошибка при обновлении/выгрузки данных.

Пропущенные количественные данные в данном случае лучше всего заполнить медианным значением по столбцу, чтобы исключить влияние выбросов данных (аномально малых и аномально больших значений). Медианное значение лучше отражает усредненное значение по анализируемой совокупности данных без эффекта "средней температуры по больнице".

**Заполнение пропусков в данных**

Заполним пропуски в столбце total_income медианным значением дохода. 

Заполнять пропуски в столбце days_employed, также как исправлять отрицательные значения в этом столбце не будем, так как этот столбец не пригодится для дальнейшего анализа. Кроме того, корректность данных в нем вызывает большие сомнения, так 340 266 дней равняется примерно 932 лет, что не может считаться реальным трудовым стажем.

Рассчитаем медианный доход

In [7]:
median_income = data[data['total_income'].isna() == False]['total_income'].median() # находим медианный доход по значениям в столбце total_income, где нет пропусков
print('Медианный доход:', int(median_income))

Медианный доход: 145017


Заполним пропуски в столбце total_income значением медианного дохода

In [8]:
data['total_income'] = data['total_income'].fillna(median_income) #заполняем пропущенные значения медианным доходом методом fillna
data['days_employed'] = '' #заполняем столбец days_employed пустой строкой
display(data.isna().sum()) # еще раз выводим пропущенные значения по столбцам

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Пропуски в данных устранены.

Заменим вещественные значения в столбце total_income целочисленными.

In [9]:
data['total_income'] = data['total_income'].astype('int') # переводим в целочисленные значения столбец total_income с помощью метода astype
data.info() # проверяем изменение типа данных в столбце total_income

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  object
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int32 
 11  purpose           21525 non-null  object
dtypes: int32(1), int64(5), object(6)
memory usage: 1.9+ MB


Тип данных в столбце total_income изменился на int (целое число).

### Поиск и обработка дубликатов

**Проверяем данные на неявные дубликаты**

Проверку будем осуществлять по каждому столбцу выводя список уникальных значений методом value_counts, таким образом также получится сделать дополнительную проверку на аномальные значения в данных.

* столбец children

In [10]:
print(data['children'].value_counts()) # проверяем первый столбец children методом value_counts, который формирует список уникальных значений и выводит частоту их повторения
print(data['children'].value_counts()/data['children'].count()) #проверяем долю аномалий

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64
 0     0.657329
 1     0.223833
 2     0.095470
 3     0.015331
 20    0.003531
-1     0.002184
 4     0.001905
 5     0.000418
Name: children, dtype: float64


Как таковых, дубликатов в столбце children, который отражает количество детей в семье заёмщика быть не может. Но благодаря данной проверке мы выявили аномальные значения в этом столбце: детей точно не может быть -1. 20 детей скорее всего тоже не является реалистичным значением.

In [11]:
data.drop(index = data[data['children'].isin([-1, 20])].index, inplace = True) 
# удаляем строчки с аномалиями методом drop с логической индексацией и параметром inplace = True для применения к текущему датасету
data = data.reset_index(drop = True) # сбрасываем индексацию после удаления строк
print(data['children'].value_counts()) # проверяем, что строчки с аномальными значениями удалены

0    14149
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64


* столбец days_employed : пропускаем как нерелевантный для исследования

* столбец dob_years

In [12]:
data['dob_years'].value_counts()

35    614
40    603
41    603
34    597
38    595
42    592
33    577
39    572
31    556
36    553
44    543
29    543
30    536
48    536
37    531
43    510
50    509
32    506
49    505
28    501
45    494
27    490
52    483
56    482
47    480
54    476
46    469
58    461
57    457
53    457
51    446
59    441
55    441
26    406
60    376
25    356
61    353
62    351
63    268
64    263
24    263
23    252
65    194
22    183
66    183
67    167
21    110
0     100
68     99
69     83
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

Присутствуют записи с аномальным значением возраста клиента 0 лет. Заполним их медианным значением возраста.

In [13]:
median_years = data['dob_years'].median() # находим медианное значение
print('Медианный возраст:', int(median_years))
data['dob_years'] = data['dob_years'].replace(0, median_years) # заменяем аномальные значение методом replace
print(data['dob_years'].value_counts()) # проверяем итоговый набор уникальных данных

Медианный возраст: 42
42    692
35    614
41    603
40    603
34    597
38    595
33    577
39    572
31    556
36    553
29    543
44    543
48    536
30    536
37    531
43    510
50    509
32    506
49    505
28    501
45    494
27    490
52    483
56    482
47    480
54    476
46    469
58    461
53    457
57    457
51    446
59    441
55    441
26    406
60    376
25    356
61    353
62    351
63    268
64    263
24    263
23    252
65    194
66    183
22    183
67    167
21    110
68     99
69     83
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64


Аномальное значение 0 лет заменено на медианное 42 года

* столбец education

In [14]:
data['education'].value_counts()

среднее                13667
высшее                  4698
СРЕДНЕЕ                  766
Среднее                  703
неоконченное высшее      665
ВЫСШЕЕ                   271
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

В столбце education присутствуют неявные дубликаты из-за использования разного регистра. Приведем все записи в этом столбце к нижнему регистру

In [15]:
data['education'] = data['education'].str.lower() # используем метод str.lower для приведения записей в столбце к нижнему регистру
data['education'].value_counts() # контрольный вывод уникальных значений в столбце

среднее                15136
высшее                  5237
неоконченное высшее      741
начальное                282
ученая степень             6
Name: education, dtype: int64

* столбец family_status

In [16]:
data['family_status'].value_counts()

женат / замужем          12302
гражданский брак          4160
Не женат / не замужем     2799
в разводе                 1189
вдовец / вдова             952
Name: family_status, dtype: int64

Особых проблем в данном столбце нет, для единообразия можно привести данный столбец также к нижнему регистру

In [17]:
data['family_status'] = data['family_status'].str.lower() # используем метод str.lower для приведения записей в столбце к нижнему регистру
data['family_status'].value_counts() # контрольный вывод уникальных значений в столбце

женат / замужем          12302
гражданский брак          4160
не женат / не замужем     2799
в разводе                 1189
вдовец / вдова             952
Name: family_status, dtype: int64

столбец gender

In [18]:
data['gender'].value_counts()

F      14154
M       7247
XNA        1
Name: gender, dtype: int64

В данном столбце отсутствуют неявные дубликаты, наличие пропуска/гендера, отличного от "F" / "M" не повлияют на результаты исследования 

* столбец income_type

In [19]:
data['income_type'].value_counts()

сотрудник          11050
компаньон           5054
пенсионер           3839
госслужащий         1453
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

С данными порядок, дубликатов нет

* столбец purpose

In [20]:
data['purpose'].value_counts()

свадьба                                   796
на проведение свадьбы                     772
сыграть свадьбу                           769
операции с недвижимостью                  673
покупка коммерческой недвижимости         661
покупка жилья для сдачи                   651
операции с жильем                         648
операции с коммерческой недвижимостью     646
жилье                                     642
покупка жилья                             641
покупка жилья для семьи                   640
недвижимость                              632
строительство собственной недвижимости    628
операции со своей недвижимостью           626
строительство жилой недвижимости          622
строительство недвижимости                620
покупка своего жилья                      619
покупка недвижимости                      619
ремонт жилью                              609
покупка жилой недвижимости                603
на покупку своего автомобиля              504
заняться высшим образованием      

Цели получения кредитов разные, однако они поддаются группировке. Создадим отдельные категории по цели получения кредита на этапе категоризации данных.

* cтолбец total_income

как таковых неявных дубликатов в этом столбце не ожидается, но нелишним будет проверить этот столбец на аномальные значения в данных

In [21]:
print('Минимальный доход', data['total_income'].min())
print('Максимальный доход', data['total_income'].max())
print('Средний доход', data['total_income'].mean())
print('Медианный доход', data['total_income'].median())

Минимальный доход 20667
Максимальный доход 2265604
Средний доход 165182.31184001494
Медианный доход 145017.0


Каких-либо аномалий для столбца total_income не выявлено.

Вначале (удаление явных дубликатов до полной обработки данных) было найдено 54 явных дубликата. Далее мы перенесли удаление явных дубликатов в конец раздела предобработки данных и количество явных дубликатов увеличилось до 72. По всей видимости, свою роль сыграло заполнение пропусков данных, обработка аномальных значений и приведение данных к единому регистру.

**Проверяем данные и удаляем явные дубликаты**

In [22]:
print('Количество дубликатов:', data.duplicated().sum()) # подсчитаем количество явных дубликатов в данных, найдено 54 дубликата

Количество дубликатов: 72


In [23]:
data = data.drop_duplicates().reset_index(drop = True) # удаляем дубликаты и сбарсываем индексы строк
print('Количество дубликатов:', data.duplicated().sum()) # проверяем, остались ли дубликаты

Количество дубликатов: 0


Проверка и обработка дубликатов завершена. Были удалены 72 явных дубликата и решена проблема с неявными дубликатами в столбце education. 

Для выявления и удаления явных дубликатов  использовались методы duplicated и drop_duplicates, а также reset_index для сброса индексов строк после удаления дубликатов.

Для выявления неявных дубликатов использовался метод value_counts, который позволяет одновременно вывести уникальные значения столбца и частотность их повторения. Для удаления неявных дубликатов использовались методы приведения записей в столбцах к нижнему регистру str.lower. Также неявные дубликаты в столбце purpose будут обработаны с помощью создания отдельного столбца с категориями цели кредита на этапе категоризации данных. 

Вероятнее всего дубликаты возникли на этапе сбора данных вследствие того, что данные вносились по разному (часть - в верхнем регистре, часть - в нижнем). Также информация о цели кредита, видимо, вносилась клиентами в свободной форме. В будущем необходимо ввести правила заполнения и сбора данных, сделать фиксированные списки для аспектов информации о клиентах, поддающихся категоризации (например, цель кредита - покупка автомобиля, покупка недвижимости, образование).

Также попутно мы проверили столбцы на аномальные значения, которые не были выявлены на ранней стадии подготовки данных.

Параметры датасета после предобработки:

In [24]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21330 entries, 0 to 21329
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21330 non-null  int64 
 1   days_employed     21330 non-null  object
 2   dob_years         21330 non-null  int64 
 3   education         21330 non-null  object
 4   education_id      21330 non-null  int64 
 5   family_status     21330 non-null  object
 6   family_status_id  21330 non-null  int64 
 7   gender            21330 non-null  object
 8   income_type       21330 non-null  object
 9   debt              21330 non-null  int64 
 10  total_income      21330 non-null  int32 
 11  purpose           21330 non-null  object
dtypes: int32(1), int64(5), object(6)
memory usage: 1.9+ MB


### Категоризация данных

Выполним категоризацию данных для удобства и наглядности дальнейшего анализа.

**Создание справочников и удаление избыточных столбцов из датасета**

Создадим новые датафреймы - справочники для хранения информации об образовании и семейном положении заемщиков. Это позволит сделать исходную таблицу с данными более лаконичной.

In [25]:
# создадим новые датафреймы путем выбора нужных столбцов из исходного датафрейма и удаления дубликатов, 
#таким образом получим таблицы - справочники

family_status = data[['family_status_id', 'family_status']].drop_duplicates().reset_index(drop = True)
education = data[['education_id', 'education']].drop_duplicates().reset_index(drop = True)

display(family_status, education) # выводим получившиеся датафреймы. 

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,не женат / не замужем


,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


После создания справочников удалим из датасета столбцы education и family status. Также удалим столбец days_employed.

In [26]:
data = data.drop(['education', 'family_status', 'days_employed'], axis = 1) # удаляем столбцы методом drop
data.head() # выводим измененный датасет

,children,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,53,1,1,F,пенсионер,0,158616,сыграть свадьбу


**Выделение категорий**

Выделим категории доходов заещиков и напишем функцию для автоматического присвоения категорий:

* 0–30000 — 'E';
* 30001–50000 — 'D';
* 50001–200000 — 'C';
* 200001–1000000 — 'B';
* 1000001 и выше — 'A'.

In [27]:
def total_income_category(total_income):  # создаем функцию для категоризации заемщиков по уровню дохода
    if 0 < total_income <= 30000:
        return 'E'
    if 30000 < total_income <= 50000:
        return 'D'
    if 50000 < total_income <= 200000:
        return 'C'
    if 200000 < total_income < 1000000:
        return 'B'
    if 1000000 < total_income:
        return 'A'

Создадим новый столбец 'total_income_category', который заполним обозначениями категорий дохода

In [28]:
data['total_income_category'] = data['total_income'].apply(total_income_category) # применяем созданную ранее функцию к значениям в столбце total_income с помощью метода apply 
data.head() # выводим обновленный датасет для проверки

,children,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C


Создадим функцию для определения категорий назначения кредита. Для определения принадлежности к той или иной категории используем поиск по подстрокам.

* ‘операции с автомобилем’, : 'авто'
* ‘операции с недвижимостью’, : 'недвиж', 'жил'
* ‘проведение свадьбы’, : 'свадьб'
* ‘получение образования’ : 'образован'


In [29]:
def purpose_category(purpose):
    if 'авто' in purpose:
        return 'операции с автомобилем'
    elif 'недвиж' in purpose or 'жил' in purpose:
        return 'операции с недвижимостью'
    elif 'свадьб' in purpose:
        return 'проведение свадьбы'
    elif 'образован' in purpose:
        return 'получение образования'

Создадим новый столбец 'purpose_category', который заполним обозначениями категорий назначения кредита

In [30]:
data['purpose_category'] = data['purpose'].apply(purpose_category) # применяем созданную ранее функцию к значениям в столбце total_income с помощью метода apply 
display(data.head()) # выводим обновленный датасет для проверки
print(data['purpose_category'].value_counts()) # на всякий случай проверим, что ко всем строкам были подобраны категории по ключевым словам (подстрокам)

,children,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы


операции с недвижимостью    10751
операции с автомобилем       4279
получение образования        3988
проведение свадьбы           2312
Name: purpose_category, dtype: int64


Предобработка данных завершена, были устранены пропуски в данных, исправлены аномальные значения, проверены и удалены явные и неявные дубликаты, а также выполнена категоризация данных. Можно переходить непосредственно к анализу данных.

## Анализ данных и решение поставленных задач

В данном разделе мы перейдем к непосредственному анализу данных и постараемся ответить на вопросы, обозначенные в начале исследования.

**1. Есть ли зависимость между количеством детей и возвратом кредита в срок?**

Сформулируем исходную гипотезу: с увеличением количества детей в семье вероятность возврата кредита в срок снижается.

Напишем функцию, которая будет строить сводную таблицу, которая отражает количество детей и наличие задолженности по кредиту  с помощью метода pivot_table. Аргументом функции будет название столбца датафрейма, который будет источником для строк сводной таблицы.

Основную информацию и значения по столбцам мы проверили на этапе предобработки данных, получения ошибок при выполнении кода не ожидается, поэтому не будем применять обработчик ошибок **try...except...** в данном проекте. 

In [31]:
def def_pivot(pivot_index):
    pivot_def = pd.pivot_table(data, values = ['debt','gender'], index = [pivot_index], aggfunc = {'debt' : 'sum', 'gender' : 'count'}) 
    # строим сводную таблицу с двумя значениями, sum по столбцу debt для подсчета количества заемщиков с задолженностью и count для подсчета общего количества заемщиков в каждой категории
    pivot_def['debt_share'] = (pivot_def['debt'] / pivot_def['gender'] * 100).round(decimals = 1) # добавляем столбец с долей должников от общего количества заемщиков
    pivot_def = pivot_def.rename(columns = {'gender' : 'total'}) # переименовываем столбец с подсчетом общего количества заемщиков в каждой категории
    return pivot_def.sort_values(by = 'debt_share', ascending = False) # выводим итоговую таблицу, отсортированную по убыванию доли заемщиков с задолженностью

В получившейся таблице children отражает количество детей в семье заемщика, debt количество заемщиков, допустивших просрочку по выплатам кредита, non-debt количество заемщиков без просрочек по кредитам и debt_share доля заещиков с просрочкой в общем количестве заемщиков.

Максимальное количество заемщиков с просрочкой не имеет детей, однако делать выводы преждевременно, так как заемщики без детей составляют самое большое количество среди заемщиков. 

Сравним доли заемщиков с просрочкой от общего количества заемщиков. На первом месте с долей 9.8% заемщики с четырьмя детьми, далее располагаются заемщики с двумя, одним и тремя детьми. Доля просрочек по кредитам среди заемщиков без детей составляет 7.5%. Среди полученной статистики из общей закономерности выбивается 9 заемщиков с пятью детьми, не имевших просрочек по кредитам. Взглянем подробнее на данные по таким заемщикам.

In [32]:
display(data[data['children'] == 5])

,children,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
3957,5,42,1,1,M,сотрудник,0,145017,на покупку своего автомобиля,C,операции с автомобилем
4373,5,36,1,0,F,компаньон,0,168460,операции с недвижимостью,C,операции с недвижимостью
7819,5,36,1,0,F,сотрудник,0,48772,операции с жильем,D,операции с недвижимостью
15697,5,31,1,0,F,сотрудник,0,77552,сделка с подержанным автомобилем,C,операции с автомобилем
15791,5,37,1,0,F,сотрудник,0,256698,покупка недвижимости,B,операции с недвижимостью
16081,5,35,1,1,F,госслужащий,0,126102,на проведение свадьбы,C,проведение свадьбы
20270,5,38,3,0,F,сотрудник,0,212545,заняться высшим образованием,B,получение образования
20652,5,35,1,0,F,компаньон,0,204241,жилье,B,операции с недвижимостью
20967,5,59,1,0,M,сотрудник,0,269068,операции со своей недвижимостью,B,операции с недвижимостью


Явных проблем и несоответствий с данными строчками не выявлено. у 7 из 9 заемщиков совокупный доход составляет более 100 тыс. руб., что косвенно подтверждает гипотезу о корректности данных. Будем считать, что данные заемщики обладают исключительно 
хорошими навыками финансового планирования и смогли вернуть свои кредиты без просрочек.

**Выводы**

Таким образом, между наличием детей и просрочками по кредиту есть зависимость. Доля должников больше среди заемщиков, имеющих детей и, соответственно, более высокие запланированные и незапланированные расходы. Однако прямой зависимости между количеством детей и долей заемщиков с задолженностью не выявлено, исходная гипотеза подтвердилась лишь частично.

**2. Есть ли зависимость между семейным положением и возвратом кредита в срок?**

Исходная гипотеза: разное семейное положение влияет на выплату кредита в срок

Построим соответствующую сводную таблицу для проверки данной гипотезы. За основу возьмем конструкцию из предыдущей задачи, в качестве строк сводной таблицы будет использоваться категория семейного положения.

In [33]:
pivot_family = def_pivot('family_status_id')  # используем уже созданную функцию def_pivot с новым аргументом family_status_id
pivot_family = pivot_family.merge(family_status, how = 'left', on = 'family_status_id') # добавляем семейное положение из ранее созданного справочника family_status
display(pivot_family)

,family_status_id,debt,total,debt_share,family_status
0,4,273,2796,9.8,не женат / не замужем
1,1,385,4133,9.3,гражданский брак
2,0,927,12261,7.6,женат / замужем
3,3,84,1189,7.1,в разводе
4,2,63,951,6.6,вдовец / вдова


**Выводы**

Самая большая доля задолженности по кредитам наблюдается у заемщиков, не состоящих в браке. При этом меньше всего проблем с возвратом кредитов имеют разведенные заемщики, а также заемщики вдовцы/вдовы. Можно предположить, что неженатые заемщики более легкомысленно подходят к взятию кредита и соблюдению сроков его погашения. В свою очередь, женатые/разведенные заемщики, а тем более заемщики-вдовы/вдовцы более осознанно подходят к такому решению и стараются не допустить возникновение задолженности.

Можно сделать вывод, что возврат кредита в срок зависит от семейного положения, гипотеза подтвердилась.

**3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

Исходная гипотеза: Заемщики с более низкими уровнями дохода склонны задерживать выплаты по кредитам, заемщики с более высокими уровнями дохода допускают меньше просрочек по кредитам.

Построим сводную таблицу с детализацией по уровню дохода

In [34]:
display(def_pivot('total_income_category')) # используем уже созданную функцию def_pivot с новым аргументом total_income_category

,debt,total,debt_share
total_income_category,,,
E,2,22,9.1
C,1353,15921,8.5
A,2,25,8.0
B,354,5013,7.1
D,21,349,6.0


**Выводы**

Полученная таблица показывает распределение заемщиков по уровню дохода, а также долю должников в каждой выделенной категории заемщиков. Категория "А" - клиенты с самым высоким уровнем дохода, соответственно, категория "Е" - клиенты с самым низкким уровнем дохода. Наибольшая доля должников в категории "Е", при этом лучше всего возвращают кредиты заемщики из категории "D", которая также характеризуется невысоким уровнем дохода.

Количество заемщиков неравномерно распределено по категориям. Количество заемщиков в категориях "E" и "А" очень низкое по сравнению с количеством заемщиков в других категориях. В случае категории "Е" это может быть вызвано редким одобрением кредита, а в случае категории "А", видимо, у обеспеченных людей реже возникает потребность в кредите. 

При этом гипотеза о том, что заемщики с более высоким уровнем дохода будут реже иметь задолженность по кредитам не подтвердилась, видимо, с ростом дохода растет и сумма кредита.  Зависимости между уровнем дохода и возвратом кредите в срок не наблюдается.

**4. Как разные цели кредита влияют на его возврат в срок?**

Исходная гипотеза: Разные цели кредита влияют на возврат кредита в срок.

Построим сводную таблицу с детализацией по цели кредита

In [35]:
display(def_pivot('purpose_category')) # используем уже созданную функцию def_pivot с новым аргументом purpose_category

,debt,total,debt_share
purpose_category,,,
операции с автомобилем,400,4279,9.3
получение образования,369,3988,9.3
проведение свадьбы,183,2312,7.9
операции с недвижимостью,780,10751,7.3


**Выводы**

Распределение заемщиков по категориям цели кредита более равномерное. Самая большая доля просрочек по выплате кредитов среди заемщиков с целью кредита "операции с автомобилем", а самая низкая среди заемщиков с целью "операции с недвижимостью". 
* Высокую долю невозвратов кредита на автомобиль в срок можно объяснить тем, что  покупка автомобиля зачастую сопряжена с дополнительными расходами (покупка резины, установка доп. оборудования и т.п.), не говоря уже о риске повреждении автомобиля в случае аварии и сопряженными с этим финансовыми затратами
* В случае получения образования также существуют риски того, что у заемщика не получится трудоустроиться в намеченный срок, что в итоге приведет к задержке выплаты кредита
* Проведение свадьбы предполагает традиционно предполагает покрытие части расходов за счет подарков гостей, бюджет свадьбы фиксирован и риски невозврата кредита в срок ниже, чем в двух предыдущих вариантах
* Операции с недвижимостью - это ответственный шаг, также часто заемщики оформляют вместе с ипотекой страховку на случай потери трудоспособности. Это может объяснять более низкий процент задержек по выплате кредитов в случае операций с недвижимостью.

Гипотеза о наличии зависимости между целью кредита и возвратом кредита в срок подтвердилась.

## Итоги исследования

Мы проанализровали статистику по возврату кредитов заемщиками для совершенствования модели кредитного скоринга.  

1. В ходе предобработки данных были выявлены и устранены ряд проблем:

* Пропуски в данных (10% от общего количества записей в 2 столбцах). Пропуски в столбце total_income были заменены на медианные значения, пропуски в столбце days_employed не повлияли на исследование.
* Аномальные значения. Аномальные значения в столбце children составили менее 1% от общего числа записей и были отброшены. Аномальные значения в столбце dob_years (возраст замемщиков) были заполнены медианным значением возраста.
* Явные и неявные дубликаты. Неявные дубликаты были устранены с помощью обработки данных, явные дубликаты были удалены из датасета, их оказалось немного (72 шт.)

2. Для удобства анализа была проведена категоризация данных:

* по уровню дохода
* по цели кредита

3. Были проверены гипотезы, сформулированные в начале исследования:

1) Есть ли зависимость между количеством детей и возвратом кредита в срок?

Наличие детей увеличивает вероятность задержки выплаты по кредиту. Но прямой взаимосвязи между количеством детей и задержкой платежа по кредиту не установлено.

2) Есть ли зависимость между семейным положением и возвратом кредита в срок?

Заемщики, не состоящие в браке склонны задерживать выплаты по кредиту по сравнению с женатыми/замужними, либо разведенными заемщиками. Самый низкая доля должников среди вдов/вдовцов. Таким образом между семейным положиением и возвратом кредита в срок существует зависимость.

3) Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Заемщики с самым низким уровнем дохода хуже остальных возвращают криедиты. Однако в дальнейшем гипотеза не нашла подтверждения, уровень дохода практически не влияет на возврат кредита в срок.

4) Как разные цели кредита влияют на его возврат в срок?

В зависимости от цели кредита меняется доля заемщиков, которые возвращают его в срок. Операции с недвижимостью характеризуется самым низким уровнем просрочек по кредитам, в то время как доля должников по операциям с автомобилем наибольшая. 

Исходная гипотеза о влиянии целей кредита на его возврат в срок подтвердилась.

4. На основе полученных результатов можно сформировать следующие рекомендации для совершенствования модели кредитного скоринга:

**Учитывать в модели:**

Цель кредита и семейное положение, так как между данными факторами и выплатой кредита в срок существует зависимость.

**Не учитывать в модели:**

Количество детей и уровень дохода. При этом можно ввести фактор наличия детей в семье, так как заемщики с наличием детей в семье более склонны к задержкам выплат по кредитам.



